In [1]:
# https://medium.com/datadriveninvestor/elons-lethal-mistake-predicting-the-stock-price-of-tesla-with-twitter-and-machine-learning-5e89282ce75f
# snscrape --max-results 100 twitter-search tesla >twitter-@tesla

In [43]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib.pyplot as mlpt
import pandas as pd
import numpy as np

from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score

from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LogisticRegression
from datetime import datetime, timedelta

import yfinance as yf

In [44]:
df = pd.read_csv('Extracted_TWEETS.csv')

In [45]:
df.head()

,created_at,text
0,2020-10-03 02:42:11,@mithunoorath @WholeMarsBlog @Tesla @elonmusk ...
1,2020-10-03 02:47:24,A new study found that the average golfer walk...
2,2020-10-03 02:45:12,@TeslaUnofficial Tesla is not just a cars/stor...
3,2020-10-03 02:48:12,@chainlink @CryptoDavid_ @Tesla Rogue: Request...
4,2020-10-03 02:47:52,@timlopez @Tesla They will do anything to push...


In [46]:
df.dtypes

created_at    object
text          object
dtype: object

In [47]:
def transformDate(date):
    return date.split(' ')[0]

In [48]:
df['created_at'] = df['created_at'].apply(transformDate)

In [49]:
df.head()

,created_at,text
0,2020-10-03,@mithunoorath @WholeMarsBlog @Tesla @elonmusk ...
1,2020-10-03,A new study found that the average golfer walk...
2,2020-10-03,@TeslaUnofficial Tesla is not just a cars/stor...
3,2020-10-03,@chainlink @CryptoDavid_ @Tesla Rogue: Request...
4,2020-10-03,@timlopez @Tesla They will do anything to push...


In [50]:
index=0
for index,row in df.iterrows():
    stre=row["text"]
    my_new_string = re.sub('[^ a-zA-Z0-9]', '', stre)
    df.at[index,'created_at'] = row["created_at"]
    df.at[index,'text'] = my_new_string
    index=index+1
    
    
df = df.groupby(['created_at'], as_index = False).agg({'text': ''.join}, Inplace=True)

In [51]:
df.head()

,created_at,text
0,2020-10-02,Dealing with a herniated disk can be excruciat...
1,2020-10-03,mithunoorath WholeMarsBlog Tesla elonmusk 28de...


In [52]:
tsla_df = yf.download('TSLA', 
                      start='2020-10-01', 
                      end='2020-10-03')
tsla_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-10-01,440.760010,448.880005,434.420013,448.160004,448.160004,50741500
2020-10-02,421.390015,439.130005,415.000000,415.089996,415.089996,71055600


In [56]:
tsla_df.reset_index(level=0, inplace=True)

In [57]:
df['Price'] = tsla_df['Close']

In [58]:
df.head()

,index,created_at,text,Price
0,0,2020-10-02,Dealing with a herniated disk can be excruciat...,448.160004
1,1,2020-10-03,mithunoorath WholeMarsBlog Tesla elonmusk 28de...,415.089996


In [59]:
dataframe = df[['Price']].copy()
dataframe["Comp"] = ''
dataframe["Negative"] = ''
dataframe["Neutral"] = ''
dataframe["Positive"] = ''

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
sentiment_i_a = SentimentIntensityAnalyzer()
for indexx, row in df.T.iteritems():
    try:
        sentence_i = unicodedata.normalize('NFKD', df.loc[indexx, 'text'])
        sentence_sentiment = sentiment_i_a.polarity_scores(sentence_i)
        dataframe.at[indexx, 'Comp'] = sentence_sentiment['compound']
        dataframe.at[indexx, 'Negative'] = sentence_sentiment['neg']
        dataframe.at[indexx, 'Neutral'] = sentence_sentiment['neu']
        dataframe.at[indexx, 'Positive'] = sentence_sentiment['pos']
    except TypeError:
        print (df.loc[indexx, 'Tweets'])
        print (indexx)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/caiquecoelho/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [60]:
dataframe.head()

,Price,Comp,Negative,Neutral,Positive
0,448.160004,-0.9136,0.246,0.754,0
1,415.089996,1,0.06,0.789,0.151
